In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lmsys-chatbot-arena/sample_submission.csv
/kaggle/input/lmsys-chatbot-arena/train.csv
/kaggle/input/lmsys-chatbot-arena/test.csv


# Initial Data Exploration

In [2]:
df = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
df.head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0


In [3]:
df.info()

# FINDING: No Null values in the data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57477 entries, 0 to 57476
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              57477 non-null  int64 
 1   model_a         57477 non-null  object
 2   model_b         57477 non-null  object
 3   prompt          57477 non-null  object
 4   response_a      57477 non-null  object
 5   response_b      57477 non-null  object
 6   winner_model_a  57477 non-null  int64 
 7   winner_model_b  57477 non-null  int64 
 8   winner_tie      57477 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 3.9+ MB


In [4]:
print("prompt: " + df.prompt[0] + '\n')
print("response_a: " + df.response_a[0] + '\n')
print("response_a type: " + str(type(eval(df.response_a[0]))) + '\n')
print("response_a length: " + str(len(eval(df.response_a[0]))))

# FINDING: Prompts and response are in string of a list object format
# FINDING: Some rows contain multiple prompts and response

prompt: ["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]

response_a: ["The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. *

# Data Pre-processing

In [5]:
import copy
df_orig = copy.deepcopy(df)

In [6]:
# Use only the first prompt-responses as input
# TODO: Do feature engineering and edit this cell
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null", "''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])
df.head()

# FINDING: Responses contain "null" values

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",1,0,0
1,53567,koala-13b,gpt-4-0613,What is the difference between marriage licens...,A marriage license is a legal document that al...,A marriage license and a marriage certificate ...,0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,explain function calling. how would you call a...,Function calling is the process of invoking or...,Function calling is the process of invoking a ...,0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,How can I create a test set for a very rare ca...,Creating a test set for a very rare category c...,When building a classifier for a very rare cat...,1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,What is the best way to travel from Tel-Aviv t...,The best way to travel from Tel Aviv to Jerusa...,The best way to travel from Tel-Aviv to Jerusa...,0,1,0


## Inspect "null" values in responses

In [7]:
print("The number of null response_a: " + str(df_orig.response_a[df_orig.response_a == "null"].count()))
print("The number of null response_b: " + str(df_orig.response_b[df_orig.response_b == "null"].count()))

# FINDING: No response is "null"

The number of null response_a: 0
The number of null response_b: 0


In [8]:
def contains_null(string):
    return True if string.find("null") != -1 else False

df_orig[df_orig.response_a.apply(contains_null)].head()

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
130,9644862,vicuna-7b,RWKV-4-Raven-14B,"[""Write me a Unity editor script that lists al...","[""Here is a sample script that lists all the A...","[""```\nusing UnityEngine;\nusing System.Collec...",1,0,0
191,15247315,gpt-3.5-turbo-0613,gpt-4-0613,"[""generate an laravel migration""]","[""To generate a Laravel migration, you can use...","[""Sure, here's a basic Laravel migration examp...",0,0,1
207,16350735,claude-instant-1,tulu-2-dpo-70b,"[""Tell me about wwiii"",""Enough""]","[""I do not have any confirmed details about a ...","[""As an AI language model, I must clarify that...",0,1,0
281,22179610,claude-2.0,codellama-34b-instruct,"[""please write me a piece of Java-Code with Ja...","[""Here is the Java code to check if a list has...","[""Sure, here is a Java code using Java Streams...",0,0,1
333,25545380,vicuna-33b,vicuna-7b,"[""How to create a database model with sqlalche...","[""SQLAlchemy is a popular Python SQL toolkit a...","[""To create a database model using SQLAlchemy,...",1,0,0


In [9]:
df_orig.loc[130, "response_a"]
# FINDING: The value only contains null as a part of response

'["Here is a sample script that lists all the AudioSources in the scene and has a way to play them:\\n```\\nusing UnityEngine;\\nusing UnityEditor;\\n\\n[ExecuteInEditMode]\\npublic class AudioSourceSelector : EditorWindow\\n{\\n    public static AudioSourceSelector current;\\n\\n    [MenuItem(\\"Tools\\/Audio Source Selector\\")]\\n    static void ShowWindow()\\n    {\\n        if (current != null)\\n        {\\n            current.Close();\\n        }\\n\\n        current = new AudioSourceSelector();\\n        current.Show();\\n    }\\n\\n    private void OnSelectAudioSource()\\n    {\\n        \\/\\/ Clear the audio sources list\\n        audioSourceList.Clear();\\n\\n        \\/\\/ Get all the AudioSources in the scene\\n        AudioSource[] audioSources = FindObjectsOfType<AudioSource>();\\n\\n        \\/\\/ Add all the AudioSources to the list\\n        foreach (AudioSource audioSource in audioSources)\\n        {\\n            audioSourceList.Add(audioSource);\\n        }\\n   